## Watch OrderBook of multiple ticker pair and multiple exchange asynchronous  
- this code runs on the `crypto` image 
- you can change the `slack_channel` to a your existing slack channel to get alert to the job
- you can use the `pipeline.yaml` to spin up a `Hyperplane service` to watch the orderbook indefinitly 

In [2]:
import warnings
warnings.filterwarnings("ignore")

import json
import os
import ccxtpro
import ccxt
import asyncio
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import traceback

import nest_asyncio
nest_asyncio.apply()

## initialize a slack notification for trades or other alerts
from hyperplane import notebook_common as nc
sh = nc.SlackHelper()

In [5]:
## parameters can be injected when spijn up the jobs

slack_channel = '#orderbook'

exchanges = {
        'kucoin': ['BTC/USDT', 'ETH/BTC', 'ETH/USDT'],
        'binance': ['BTC/USDT', 'ETH/USDT','XRP/USDT']
    }

## Watch multiple orderbooks 
- available apis : watch_trades, watch_ticker, watch_orderbook, watch_market, watchOHLCV

In [22]:
orderbooks = {}

def handle_all_orderbooks(orderbooks):
    """your function to use the OrderBooks"""
#     print('We have the following orderbooks:')
    for exchange_id, orderbooks_by_symbol in orderbooks.items():
        for symbol in orderbooks_by_symbol.keys():
            orderbook = orderbooks_by_symbol[symbol]
            print(ccxtpro.Exchange.iso8601(orderbook['timestamp']), exchange_id, symbol, 
                  orderbook['asks'][0], orderbook['bids'][0])
            # print(orderbook)

In [23]:
async def loop(asyncio_loop, exchange_id, symbol):
    exchange = getattr(ccxtpro, exchange_id)({
        'enableRateLimit': True,
        'asyncio_loop': asyncio_loop,
    })
    
    output_parent_path = f"gs://{os.environ['HYPERPLANE_GCP_BUCKET']}/data/crypto_trading/{exchange_id}"
    logs = []
    
    starttime = time.time()
    starttime_h = datetime.fromtimestamp(starttime).strftime("%Y-%m-%d %I:%M:%S")
    sh.post_message(json.dumps({'starttime_h': "test"}), channel=slack_channel)

    while True:
        try:
            orderbook = await exchange.watch_order_book(symbol, limit = 10)
            orderbooks[exchange.id] = orderbooks.get(exchange.id, {})
            orderbooks[exchange.id][symbol] = orderbook
            print('===========================================================')
            # print(type(orderbooks), orderbooks.keys())
            ## code for profit calculation and order 

            sh.post_message(json.dumps({'orderbooks': orderbooks}), channel=slack_channel)
            
            handle_all_orderbooks(orderbooks)
            
        except Exception as e:
            print(str(e))
            # raise e  # uncomment to break all loops in case of an error in any one of them
            # break  # uncomment to break just this one loop if it fails
    await exchange.close()

In [24]:
async def main(asyncio_loop):
    symbols = [f"{t}/{base_ticker}" for t in trade_ticker]
    print('symbols', symbols)
    exchanges = {
        exchange: symbols,
    }
    loops = [loop(asyncio_loop, exchange_id, symbol) for exchange_id, symbols in exchanges.items() for symbol in symbols]
    await asyncio.gather(*loops)


if __name__ == '__main__':
    asyncio_loop = asyncio.get_event_loop()
    asyncio_loop.run_until_complete(main(asyncio_loop))

symbols ['BTC/USDT', 'ETH/USDT']
None binance ETH/USDT [2587.04, 9.3855] [2587.03, 0.0386]
None binance ETH/USDT [2587.04, 9.3855] [2587.03, 0.0386]
None binance BTC/USDT [37870.02, 1.86727] [37870.01, 1e-05]
2022-01-29T20:18:27.411Z binance ETH/USDT [2587.04, 9.3855] [2587.03, 0.0386]
None binance BTC/USDT [37870.02, 1.86727] [37870.01, 1e-05]
2022-01-29T20:18:27.411Z binance ETH/USDT [2587.04, 9.3855] [2587.03, 0.0386]
2022-01-29T20:18:27.411Z binance BTC/USDT [37870.02, 1.09138] [37870.01, 0.21132]
